# Data Masking Demo with Purview
We will mask an ADLS2 dataset that is available within Purview
# Pre-requisites
We will use pyapacheatlas to search and extract the classifications.  
Code examples: 
- https://github.com/wjohnson/pyapacheatlas/blob/master/samples/CRUD/read_search_by_name.py
- https://www.rakirahman.me/purview-api-with-synapse/#pipeline-walkthrough

You will need an SPN that has been given "Data Source Administrator" on the root Purview collection.

In [0]:
%pip install pyapacheatlas

Python interpreter will be restarted.
Collecting pyapacheatlas
 Downloading pyapacheatlas-0.9.0-py3-none-any.whl (65 kB)
Collecting openpyxl>=3.0
 Downloading openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
Requirement already satisfied: requests>=2.0 in /databricks/python3/lib/python3.8/site-packages (from pyapacheatlas) (2.25.1)
Collecting et-xmlfile
 Downloading et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.0->pyapacheatlas) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.0->pyapacheatlas) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.0->pyapacheatlas) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.0->pyapacheatlas) (2.10)
Installing collected packages: et-xmlfile, openpyxl, pyapacheatlas
Successfully installed et-xmlfile-1.1.0 openpyxl-3.0.9 pyapacheatlas-0.9.0
WARNING: You are using pip version 21.0.1; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0b96f020-bada-4b4a-a544-08ef23858cfa/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
%pip install jmespath

Python interpreter will be restarted.
Requirement already satisfied: jmespath in /databricks/python3/lib/python3.8/site-packages (0.10.0)
WARNING: You are using pip version 21.0.1; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-0b96f020-bada-4b4a-a544-08ef23858cfa/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
dbutils.widgets.text(
  'notebook_settings_param',
  '''{ 
  "data": { 
  "config_secretScope": "kv-mdp", 
  "config_KeyVaultName": "kv-mdp", 
  "config_rawPath": "1-rawzone/jsontoparquet", 
  "config_curatedPath": "2-curatedzone/jsontoparquettodelta", 
  "kv_ADLSSourceStorageAccountURL": "yaaf-storage-account", 
  "kv_ServicePrincipalAppID": "dbwsp-mdp-appid", 
  "kv_ServicePrincipalAppKey": "dbw-mdp-secretvalue", 
  "kv_TenantID": "dbwsp-mdp-tenantid" 
  }
  }'''
)

In [0]:
import json
from pprint import pprint

settings = dbutils.widgets.get('notebook_settings_param')
json_settings = json.loads(settings)
data_settings = json_settings['data']
pprint(data_settings)

key_vault = data_settings['config_KeyVaultName']
secret_scope = data_settings['config_secretScope']
storage_account = data_settings['kv_ADLSSourceStorageAccountURL']

tenant_id = dbutils.secrets.get(scope=secret_scope,key=data_settings['kv_TenantID'])

app_id = dbutils.secrets.get(scope=secret_scope,key=data_settings['kv_ServicePrincipalAppID'])
app_key = dbutils.secrets.get(scope=secret_scope,key=data_settings['kv_ServicePrincipalAppKey'])

config_raw_path = json_settings['data']['config_rawPath']
config_curated_path = json_settings['data']['config_curatedPath']

client_id = dbutils.secrets.get(scope=secret_scope,key="pviewsp-mdp-clientid")
client_secret = dbutils.secrets.get(scope=secret_scope, key="pviewsp-mdp-secretvalue")

# constant variables
purview_name = "pview-mdp"

{'config_KeyVaultName': 'kv-mdp',
 'config_curatedPath': '2-curatedzone/jsontoparquettodelta',
 'config_rawPath': '1-rawzone/jsontoparquet',
 'config_secretScope': 'kv-mdp',
 'kv_ADLSSourceStorageAccountURL': 'yaaf-storage-account',
 'kv_ServicePrincipalAppID': 'dbwsp-mdp-appid',
 'kv_ServicePrincipalAppKey': 'dbw-mdp-secretvalue',
 'kv_TenantID': 'dbwsp-mdp-tenantid'}

In [0]:
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from requests import request
# import json
import jmespath
# from pprint import pprint


oauth = ServicePrincipalAuthentication(tenant_id=tenant_id,
                                       client_id=client_id,
                                       client_secret=client_secret
                                      )

# Create a client to connect to your service.
client = PurviewClient(account_name=purview_name,
                       authentication=oauth
                      )

In [0]:
# can search for particular classification if you wanted to
search = client.search_entities("ip address")
all_ents = [ent for ent in search 
#             if ent["entityType"] == "azure_datalake_gen2_resource_set"
           ]

# show the first entity
all_ents[0]

Out[22]: {'owner': None,
 'qualifiedName': 'https://adlsmdp.dfs.core.windows.net/scm-releases/scm-latest-fnapp-mdp.zip',
 'entityType': 'azure_datalake_gen2_path',
 'name': 'scm-latest-fnapp-mdp.zip',
 'description': None,
 'term': [],
 'id': 'ce0fdf63-e31a-4d3b-921d-fa6c09b08aa2',
 'label': [],
 'classification': [],
 'collectionId': 'pview-mdp',
 'assetType': ['Azure Data Lake Storage Gen2'],
 '@search.highlights': {'name': ['scm-latest-fnapp-mdp.<em>zip</em>']},
 '@search.score': 212.56783}

In [0]:
# our main example will search for all ADLS2 assets
search = client.search_entities("azure_datalake_gen2_resource_set")
all_entities = [ent for ent in search]

# we'll focus on just one entity for now
entity = all_entities[0]
entity

Out[23]: {'owner': None,
 'qualifiedName': 'https://adlsmdp.dfs.core.windows.net/data/bronze_raw/customer/CustomerExtract{N}.csv',
 'entityType': 'azure_datalake_gen2_resource_set',
 'name': 'CustomerExtract',
 'description': None,
 'term': [],
 'id': 'eac35964-dd71-4b2e-9a07-331ac6f110bc',
 'label': [],
 'classification': ['MICROSOFT.MISCELLANEOUS.IPADDRESS',
 'MICROSOFT.PERSONAL.AGE',
 'MICROSOFT.GOVERNMENT.CITY_NAME',
 'MICROSOFT.PERSONAL.IPADDRESS',
 'MICROSOFT.PERSONAL.NAME',
 'MICROSOFT.GOVERNMENT.COUNTRY_NAME',
 'MICROSOFT.PERSONAL.EMAIL'],
 'collectionId': 'pview-mdp',
 'assetType': ['Azure Data Lake Storage Gen2'],
 '@search.highlights': {'entityType': ['<em>azure_datalake_gen2_resource_set</em>']},
 '@search.score': 174.06454}

In [0]:
def get_schema_with_classification(entity, client):
  """
  function to retreive the schema with classification from purview
  
  :param dict entity: the entity from purview search
  :param pyapacheatlas.core.PurviewClient client: purview client for getting entities
  
  :return: qualified name of entity and schema of entity
  :rtype: list(str, list)
  """
  full_entity = client.get_entity(entity["id"])["entities"][0]
  qualified_name = full_entity["attributes"]["qualifiedName"]
  print(qualified_name)

  # API response is different based on the asset
  if qualified_name.endswith(".json"):
      print("json attached schema")
      schema_guid = jmespath.search("relationshipAttributes.attachedSchema[0].guid", full_entity)
  else:
      print("tabular schema")
      schema_guid = jmespath.search("relationshipAttributes.tabular_schema.guid", full_entity)

#   print(schema_guid)
  
  # Retrieve the schema based on the guid match
  full_schema = client.get_entity(schema_guid)
  parsed_schema = jmespath.search("[referredEntities.*.[attributes.name, classifications[0].[typeName][0]]]", full_schema)[0]

  # we only want the columns with classifications for hasing
  columns_with_classifications = [col for col in parsed_schema if col[1]]

  return qualified_name, columns_with_classifications

In [0]:
adls_path, classification_columns = get_schema_with_classification(entity, client)
pprint(classification_columns)

hash_columns = [col[0] for col in classification_columns]
print("=== === ===")
print("=== HASH COLUMNS ===")
pprint(hash_columns)

https://adlsmdp.dfs.core.windows.net/data/bronze_raw/customer/CustomerExtract{N}.csv
tabular schema
[['customerid', 'MICROSOFT.PERSONAL.AGE'],
 ['city', 'MICROSOFT.GOVERNMENT.CITY_NAME'],
 ['first_name', 'MICROSOFT.PERSONAL.NAME'],
 ['last_name', 'MICROSOFT.PERSONAL.NAME'],
 ['country', 'MICROSOFT.GOVERNMENT.COUNTRY_NAME'],
 ['ip_address', 'MICROSOFT.PERSONAL.IPADDRESS'],
 ['email', 'MICROSOFT.PERSONAL.EMAIL']]
=== === ===
=== HASH COLUMNS ===
['customerid',
 'city',
 'first_name',
 'last_name',
 'country',
 'ip_address',
 'email']

In [0]:
import pyspark.sql.functions as F

split_path = adls_path.split("/")
path_ext = adls_path.split(".")[-1]
storage_account_dfs = split_path[2]
print(storage_account_dfs)
container_name = split_path[3]
extensions_to_check = ["csv", "json", "parquet"]

# direct access
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_dfs}", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_dfs}", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_dfs}", app_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_dfs}", app_key)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_dfs}", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

mount_point = "abfss://"
if path_ext in extensions_to_check:
  data_path = "/".join(adls_path.split("/")[4:-1])
else:
  data_path = "/".join(adls_path.split("/")[4:])

# mount_point = "/mnt/data"
# data_path = "/".join(adls_path.split("/")[4:-1])

# abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/<directory-name>
full_path = f"{mount_point}{container_name}@{storage_account_dfs}/{data_path}"
hash_path = f"{full_path}_hashed"

print(full_path)

df = spark.read.csv(full_path, header=True)

# # hashing the columns with classfications
hash_df = (df
           .withColumn("sha256", F.sha2(F.concat_ws("||", *hash_columns), 256))
           .drop(*hash_columns)
          )

display(hash_df)

adlsmdp.dfs.core.windows.net
abfss://data@adlsmdp.dfs.core.windows.net/bronze_raw/customer

Title,username,gender,job_title,state,street_address,order_no,sha256
Mr,adugget0,Female,Help Desk Technician,null,48672 Burrows Parkway,727464,3ccac8f19dee688f3bcc31c0121a409436c171f7b912243e0a46c49cf4da00c3
Dr,memlyn1,Genderfluid,Desktop Support Technician,Aveiro,8 Derek Court,844962,52927a15755d5c5068fc84d7d0ce0b1e66afe98630d1d8557772250a5fbfd424
Honorable,khaselup2,Polygender,Quality Control Specialist,null,89767 Eagan Point,340381,15e5ff2d5002ef862d091e9066820ccb08144e9c67c63717ae81634d32d9f2d1
Rev,pgianetti3,Male,Chemical Engineer,null,085 Westridge Drive,908495,240992e98ccb7c461ba347d7f1af562a4f5263ee31e8bc1aa46493866d687c08
Dr,fthirtle4,Non-binary,null,null,3808 Stone Corner Parkway,514228,57b09a17b5e32e127e2c2687b45011bd2aeb966ff196ca3405d936df21107b67
Ms,lcallar5,Genderfluid,Web Developer I,null,99 Ramsey Avenue,488909,091b333c68b7b60daefc7eebb4c949370ff71011403ab4c66e295147e9478514
Mr,kkeal6,Female,Software Test Engineer III,RhÃ´ne-Alpes,1 Oriole Pass,509157,6f39bd03c92aaeeeecc03ed0f420f840b28ad83eff124fd73997ec50fba6aedb
Mrs,odebiasi7,Genderfluid,Senior Editor,null,2 Garrison Circle,928768,ef75b676bb023a055f8e9856d7b171056a58e431914a62521c938252b9859062
Rev,ocraggs8,Polygender,null,null,2 Blue Bill Park Crossing,224644,3bd4aab2743def0e4c45a2b1af11d9d5eecffe3195f5acac0ef323833cfaad10
Dr,mcurnick9,Non-binary,Community Outreach Specialist,Louisiana,2057 Pierstorff Hill,266806,afeb9ee3829a099c063906ace49a3e16e756ff7d35d25cba6d139298e7e53713


In [0]:
hash_df.write.mode("overwrite").csv(hash_path, header=True)

In [0]:
final_df = spark.read.csv(hash_path, header=True)
display(final_df)

Title,username,gender,job_title,state,street_address,order_no,sha256
Mr,adugget0,Female,Help Desk Technician,null,48672 Burrows Parkway,727464,3ccac8f19dee688f3bcc31c0121a409436c171f7b912243e0a46c49cf4da00c3
Dr,memlyn1,Genderfluid,Desktop Support Technician,Aveiro,8 Derek Court,844962,52927a15755d5c5068fc84d7d0ce0b1e66afe98630d1d8557772250a5fbfd424
Honorable,khaselup2,Polygender,Quality Control Specialist,null,89767 Eagan Point,340381,15e5ff2d5002ef862d091e9066820ccb08144e9c67c63717ae81634d32d9f2d1
Rev,pgianetti3,Male,Chemical Engineer,null,085 Westridge Drive,908495,240992e98ccb7c461ba347d7f1af562a4f5263ee31e8bc1aa46493866d687c08
Dr,fthirtle4,Non-binary,null,null,3808 Stone Corner Parkway,514228,57b09a17b5e32e127e2c2687b45011bd2aeb966ff196ca3405d936df21107b67
Ms,lcallar5,Genderfluid,Web Developer I,null,99 Ramsey Avenue,488909,091b333c68b7b60daefc7eebb4c949370ff71011403ab4c66e295147e9478514
Mr,kkeal6,Female,Software Test Engineer III,RhÃ´ne-Alpes,1 Oriole Pass,509157,6f39bd03c92aaeeeecc03ed0f420f840b28ad83eff124fd73997ec50fba6aedb
Mrs,odebiasi7,Genderfluid,Senior Editor,null,2 Garrison Circle,928768,ef75b676bb023a055f8e9856d7b171056a58e431914a62521c938252b9859062
Rev,ocraggs8,Polygender,null,null,2 Blue Bill Park Crossing,224644,3bd4aab2743def0e4c45a2b1af11d9d5eecffe3195f5acac0ef323833cfaad10
Dr,mcurnick9,Non-binary,Community Outreach Specialist,Louisiana,2057 Pierstorff Hill,266806,afeb9ee3829a099c063906ace49a3e16e756ff7d35d25cba6d139298e7e53713
